# Extra Credit Homework Assignment: 
# Model Parameter Tuning

As in the previous assignments, in this homework assignment you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).

This assignment will continue to utilize a copy of the extracted feature dataset we used in Homework 5. Recall that the dataset has been processed by performing log, z-score and range scaling. We continuing to use more than one partition worth of data, so for the scaling, the mean, standard deviation, minimum, and maximum were calculated using data from both partitions so that a global scaling can be performed on each partition. 


---

## Step 1: Downloading the Data

This assignment will continue to use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB) for a training set and [Partition 2](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/TCRPUD) as a testing set. 

---

For this assignment, cleaning, transforming, and normalization of the data has been completed using both partitions to find the various minimum, maximum, standard deviation, and mean values needed to perform these operations. Recall from lecture that we should not perform these operations on each partition individually, but as a whole as there may(will) be different values for these in different partitions. 

For example, if we perform simple range scaling on each partition individually and we see a range of 0 to 100 in one partition and 0 to 10 in another. After individual scaling the values with 100 in the first would be mapped to 1 just like the values that had 10 in the second. This can cause serious performance problems in your model, so I have made sure that the normalization was treated properly for you. 

Below you will find the full partitions and `toy` sampled data from each partition, where only 20 samples from each of our 5 classes have been included in the data.  

#### Full
- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Full Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition2ExtractedFeatures.csv)

#### Toy
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition2ExtractedFeatures.csv)

Now that you have the two files, you should load each into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

---

### Evaluation Metric

As was done in Homework 5 and 6, for each of the models we evaluate in this assignmnet, you will calculate the True Skill Statistic score using the test data from Partition 2 to determine which model performs the best for classifying the positive flaring class.

    True skill statistic (TSS) = TPR + TNR - 1 = TPR - (1-TNR) = TPR - FPR

Where:

    True positive rate (TPR) = TP/(TP+FN) Also known as recall or sensitivity
    True negative rate (TNR) = TN/(TN+FP) Also known as specificity or selectivity
    False positive rate (FPR) = FP/(FP+TN) = (1-TNR) Also known as fall-out or false alarm ratio


**Recall**

    True positive (TP)
    True negative (TN)
    False positive (FP)
    False negative (FN)
    
See [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for more information.

Below is a function implemented to provide your score for each model.

---

In [1]:
%matplotlib inline
import os
import itertools
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoLars
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans

from sklearn.svm import SVC
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def calc_tss(y_true, y_predict):
    '''
    Calculates the true skill score for binary classification based on the output of the confusion
    table function
    
        Parameters:
            y_true   : A vector/list of values that represent the true class label of the data being evaluated.
            y_predict: A vector/list of values that represent the predicted class label for the data being evaluated.
    
        Returns:
            tss_value (float): A floating point value (-1.0,1.0) indicating the TSS of the input data
    '''
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    #print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    
    return tp_rate - fp_rate

---

In addition to the TSS, you will be asked to also calculate the Heidke Skill Score (HSS) to see how much better your model performs than a random forecast.  

Below is a function implemented to provide your score fore each model.

---

In [3]:
def calc_hss(y_true, y_predict):
    '''
    Calculates the Heidke Skill Score for binary classification based on the output of the confusion
    table function.
    
    The HSS measures the fractional improvement of the forecast over the standard forecast.
    The "standard forecast" is usually the number correct by chance or the proportion 
    correct by chance.
    
        Parameters:
            y_true   : A vector/list of values that represent the true class label of the data being evaluated.
            y_predict: A vector/list of values that represent the predicted class label for the data being evaluated.
    
        Returns:
            hss_value (float): A floating point value (-inf,1.0) indicating the HSS of the input data. 
                Negative values indicate that the chance forecast is better, 0 means no skill, and a perfect forecast obtains a HSS of 1.
    '''
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    #print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    P = float(TP + FN)
    N = float(FP + TN)
    numerator = 2*((TP * TN) - (FN * FP))
    denominator = P*(FN + TN) + N*(TP + FP)
    
    return numerator/denominator

---

As in the previous assignments, we will be utilizing a binary classification of our 5 class dataset. So, below is the helper function to change our class labels from the 5 class target feature to the binary target feature. The function is implemented to take a dataframe (e.g. our `abt`) and prepares it for a binary classification by merging the `X`- and `M`-class samples into one group, and the rest (`NF`, `B`, and `C`) into another group, labeled with `1`s and `0`s, respectively.

---

In [4]:
def dichotomize_X_y(data: pd.DataFrame):
    """
    dichotomizes the dataset and split it into the features (X) and the labels (y).
    
    :return: two np.ndarray objects X and y.
    """
    data_dich = data.copy()
    data_dich['lab'] = data_dich['lab'].map({'NF': 0, 'B': 0, 'C': 0, 'M': 1, 'X': 1})
    y = data_dich['lab']
    X = data_dich.drop(['lab'], axis=1)
    return X.values, y.values

---

### Reading the partitions

In [5]:
data_dir = '/Users/npunjani/Documents/Fund_Data_Science/data'
data_file = "normalized_partition1ExtractedFeatures.csv"
data_file2 = "normalized_partition2ExtractedFeatures.csv"

In [6]:
abt = pd.read_csv(os.path.join(data_dir, data_file))
abt2 = pd.read_csv(os.path.join(data_dir, data_file2))

---

### Run Undersampling and Feature Selection

---

In [7]:
def perform_under_sample_clust(data:DataFrame)->DataFrame:
    grouped = data.groupby('lab')
    min_group = grouped.size().min()
    result_list = []
    for name, group in grouped:
        if group.shape[0] > min_group:
            size = min_group
            a = [name]*size 
            df = DataFrame(a, columns=['lab'])
            grp_feats_df = group.copy().drop(['lab'], axis=1)
            grp_feats = grp_feats_df.values
            km = KMeans(n_clusters=size)
            km.fit(grp_feats)
            clstrs = km.cluster_centers_
            df2 = DataFrame(clstrs, columns=grp_feats_df.columns)
            df = df.join(df2)
            result_list.append(df)
        else:
            result_list.append(group)
    sampled = pd.concat(result_list, axis=0)
    return sampled

In [8]:
numFeat = 20
thresh = 0.5

abt3 = abt.loc[abt['R_VALUE_median']> thresh ]
# Split the target and descriptive features for Partition 1 into two 
# different DataFrame objects
df_labels = abt3['lab'].copy()
df_feats = abt3.copy().drop(['lab'], axis=1)

# Split the target and descriptive features for Partition 2 inot two
# different DataFrame Objects
df_test_labels = abt2['lab'].copy()
df_test_feats = abt2.copy().drop(['lab'], axis=1)

# Do feature selection
feats1 = SelectKBest(f_classif, k=numFeat).fit(df_feats, df_labels)

In [9]:
# Construct a training dataset from Partition 1 with only the selected descriptive 
# features and the target feature
df_selected_feats1 = df_feats.loc[:, feats1.get_support()]
df_train_set1 = pd.concat([df_labels, df_selected_feats1], axis=1)

# Construct a testing dataset from Partition 2 with only the selected descriptive
# features and the target feature
df_test_selected_feats1 = df_test_feats.loc[:, feats1.get_support()]
df_test_set1 = pd.concat([df_test_labels, df_test_selected_feats1], axis=1)

In [10]:
sampled1 = perform_under_sample_clust(df_train_set1)

dicotomized_train = dichotomize_X_y(sampled1)
dicotomized_test = dichotomize_X_y(df_test_set1)

---
### Q1 (25 points)

Like in Q10 of HW6, this question will be utilizing the filtered and sampled datasets that was constructed above. For this question, you will again train your models on the feature selected data that had the instances below our thrshold filtered out and then had sampling by clustering performed on it. 

You will again be constructing an [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model on each one of the different datasets. Like before, you should set the `class_weight` to `balanced` when you construct your models. You will again evaluate several different settings for the regularization parameter `C`, and kernel coefficient `gamma`. The `kernel` paramter will be constant on `rbf`. 

For each of the settings, you should train the model on your training data, then test it on the testing data with the same set of selected descriptive features. You will then calculate both the TSS and HSS scores. You should store the results of your experiments for each setting in a DataFrame for later questions.  Each entry of the DataFrame should contain 'C', 'gamma', 'tss', 'hss', 'tp', 'tn', 'fp', 'fn' for each `C` and `gamma` setting you evaluate.

**Note:** The testing data has the samples in it that are below our threshold value, so you will first need to filter those out of the data you plan to pass to your model for testing. However, you still want those instances included in the calculation of the TSS and HSSS. So, your groud truth `lab` data should include all the instances in partition 2. You will need to concatenate a vector with all zeros in it to the match the labels you partitioned from the model testing data. 

Let's give you a representation of that:
    
    labels_from_data = [labels for samples > threshold] + [labels for samples <= threshold]
    predict_labels = [labels from the model on > thrshold samples] + [0s the length of samples <= threshold]



In [ ]:
thresh = 0.5
c_vals = np.arange(0.1, 1, 0.1)
gamma_vals = np.arange(0.1, 1, 0.1)
temp = [c_vals, gamma_vals]
params = list(itertools.product(*temp))
cols = ['C', 'gamma', 'tss', 'hss', 'tp', 'tn', 'fp', 'fn']

In [ ]:
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

---
### Q2 (25 points)

In this question you will be plotting your `tss` and `hss` scores in a 3D surface plot so you can investigate what ranges of values you might want to look into further. You will use [matplotlib.pyplot.figure](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.figure.html) as your plotting canvas. 

* You should set the `figsize` parameter to some set of values, maybe `(10,10)`.

* You should [add_subplot](https://matplotlib.org/3.3.4/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure.add_subplot) of `projection` type `3d` to your figure object to get the axes.

* You should set the axes labels using the [set_xlabel](https://matplotlib.org/3.3.4/api/_as_gen/matplotlib.axes.Axes.set_xlabel.html#matplotlib.axes.Axes.set_xlabel) for the x axis and the correspondig set methods for the remaining y and z axes.

* You will then use the [plot_trisurf](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html#tri-surface-plots) to plot your 3d surface using `C`, `gamma`, and `tss` as your `x`, `y`, and `z` axes in one plot.

* Make a second plot with the only change being `hss` instead of `tss`

**Note:** I have imported [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) as `cm` above. You should use this to set the `cmap` parameter of your plots and make them look nice. 

In [ ]:
features_to_look_at = ["C", "gamma", "tss"]
features_to_look_at2 = ["C", "gamma", "hss"]

In [ ]:
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

---
### Q3 (12.5 points)

From the plots in Q2 we see that the best performance is in a range where `C` is around 0.1. So, lets repeat the experiments from Q1 but make it a more granular exploration of the range between 0 and 0.1 for `C`. Make sure to save your results to a DataFrame again, we will plot them again in the next question. 

In [ ]:
thresh = 0.5
c_vals = np.arange(0.01, 0.1, 0.01)
gamma_vals = np.arange(0.1, 1, 0.1)
temp = [c_vals, gamma_vals]
params = list(itertools.product(*temp))
cols = ['C', 'gamma', 'tss', 'hss', 'tp', 'tn', 'fp', 'fn']

In [ ]:
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

---
### Q4 (12.5 points)

Repeat the plotting done in Q2, but using the results from Q3 to see how the `tss` and `hss` change as we zero in on a potential area we want to use for our model.

In [ ]:
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

---
### Q5(15 points)

We seem to have found a good set of values for `C` but `tss` and `hss` seem like they might increase if we investigate further on `gamma` so, do this.  Run the same values of `C` but expand the values of `gamma` to the range and increments below.  Then plot your results again.

In [ ]:
thresh = 0.5
c_vals = np.arange(0.01, 0.04, 0.01)
gamma_vals = np.arange(0.1, 5, 0.1)
temp = [c_vals, gamma_vals]
params = list(itertools.product(*temp))
cols = ['C', 'gamma', 'tss', 'hss', 'tp', 'tn', 'fp', 'fn']

In [ ]:
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

---
### Q6(10 points)

It looks like we found a reasonable set of values around `gamma` equals 4 to 5 and `C` between 0.01 and 0.03.  Find the `C` and `gamma` in between these ranges from the experiments in Q5  that `jointly` maximise the two values. Meaning their sum is maximized.  Then print out the values of `C` and `Gamma`

In [ ]:
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------